In [2]:
from qiskit import *

from numpy import pi

# Qual é essa porta lógica quântica II - o Retorno

Você receberá uma operação que implementa uma transformação unitária em um único qubit: ela poderá ser ou a porta Y (com a possibilidade de uma fase global extra de -1) ou a sequência de portas Pauli Z e Pauli X (com a porta Z sendo aplicada primeiro e a X depois; com a possibilidade de uma fase global extra de -1).

Seu objetivo é identificar qual das operações foi usada apenas olhando para o resultado de sua aplicação.

Você deve usar o template da célula abaixo para apresentar sua solução implementando a função `qual_porta`. Nela, você deve retornar um número entre 0 e 3, apresentando qual é a porta lógica quântica passada no argumento `U`. Você deve retornar 

* 0 quando `U` for a porta Y,
* 1 quando `U` for a porta -XZ,
* 2 quando `U` for a porta -Y e
* 3 quando `U` for a porta XZ.

Você pode usar qualquer operação quântica para descobrir qual é a porta. O objetivo é usar seus conhecimentos de computação quântica para resolver este desafio, por isso, qualquer solução que não use computação quântica não será avaliada.


**Importante:** Essa questão deve ser resolvida usando o Qiskit. Qualquer solução que não use a ferramenta será desconsiderada.

In [3]:
def run(cir, backend = "qasm_simulator"):
    be = BasicAer.get_backend(backend)
    job = execute(cir, be, shots=1)
    resultado = job.result()
    contagem = resultado.get_counts()
    return contagem.popitem()[0]


In [7]:
def qual_porta(U : QuantumCircuit) -> int:
    
    q = QuantumRegister(2, 'q')
    c = ClassicalRegister(2, 'c')
    circuit = QuantumCircuit(q, c)

    #estado de bell
    circuit.h(q)
    circuit.cz(q[0],q[1])


    #inserção de U 
    circuit.barrier(q[0], q[1])
    
    circuit.y(q[0])
    circuit.barrier(q[0], q[1])

    # inversão sobre a média 
    circuit.h(q[0])
    circuit.h(q[1])
    circuit.x(q[0])
    circuit.x(q[1])
    circuit.cz(q[0], q[1])
    circuit.x(q[0])
    circuit.x(q[1])
    circuit.h(q[0])
    circuit.h(q[1])
    
    circuit.measure(q[0], c[0])
    circuit.measure(q[1], c[1])
    
    return int(run(circuit))

> Não edite a partir daqui

## Teste sua solução 

Você pode usar o código abaixo para testar sua solução. Lembrando que, caso sua solução chegue ao resultado correto sem o uso de computação quântica, ela será desconsiderada, 

In [8]:
from random import shuffle

class Unitary:
    def __init__(self, num):
        self.qc = QuantumCircuit(1)
        if num == 0:
            self.Y()
        elif num == 1:
            self.mZX()
        elif num == 2:
            self.mY()
        elif num == 3:
            self.ZX()
    
    def Y(self):
        self.qc.y(0)

    def ZX(self):
        self.qc.z(0)
        self.qc.x(0)

    def mY(self):
        self.qc.z(0)
        self.qc.x(0)
        self.qc.y(0)
        self.qc.z(0)
        self.qc.x(0)

    def mZX(self):
        self.qc.x(0)
        self.qc.z(0)
 
     
def testar_solucao():
    tests = \
        [(Unitary(0), 0) for _ in range(10)] + \
        [(Unitary(1), 1) for _ in range(10)] + \
        [(Unitary(2), 2) for _ in range(10)] + \
        [(Unitary(3), 3) for _ in range(10)] 
    shuffle(tests)
    ok = 0
    for U, val in tests:
        solucao = qual_porta(U.qc)
        if val == solucao:
            ok += 1
    
    print(f"Solução {int((ok/len(tests))*100)}% correta")

In [9]:
testar_solucao()